<a href="https://colab.research.google.com/github/Ahtesham519/Genrative_Deep_learning_v2_2023/blob/main/DCGAN_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%load_ext autoreload
%autoreload 2
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras import (
    layers,
    models,
    callbacks,
    losses,
    utils,
    metrics,
    optimizers,
)


#0. Parameters

In [ ]:
IMAGE_SIZE = 64
CHANNELS = 1
BATCH_SIZE = 128
Z_DIM = 100
EPOCHS = 300
LOAD_MODEL = False
ADAM_BETA_1 = 0.5
ADMA_BETA_2 = 0.999
LEARNING_RATE = 0.0002
NOISE_PARAM = 0.1


#1 Prepare the data

In [ ]:
train_data = utils.image_dataset_from_directory(
    "/app/data/lego-brick-images/dataset/",
    labels = None,
    color_mode = "grayscale",
    image_size = (IMAGE_SIZE, IMAGE_SIZE),
    batch_size = BATCH_SIZE,
    shuffle = True,
    seed = 42,
    interpolation = "bilinear",
)

In [ ]:
def preprocess(img):
  """
  Normalize and reshape the images
  """
  img = (tf.cast(img, "float32")- 127.5)/ 127.5
  return img

train = train_data.map(lambda x: preprocess(x))

In [ ]:
train_sample = sample_batch(train)

#Build the GAN

In [ ]:
discriminator_input = layers.Input(shape=(IMAGE_SIZE, IMAGE_SIZE , CHANNELS))
x = layers.Conv2D(64, kernel_size = 4, strides =2 , padding = "same", use_bias = False)(
    discriminator_input
)
x = layers.LeakyReLU(0.2)(x)
x = layers.Dropout(0.3)(x)
x = layers.Conv2D(
    128, kernel_size = 4, strides = 2, padding="same", use_bias = False
)(x)
x = layers.BatchNormalization(momentum = 0.9)(x)
x = layers.LeakyReLU(0.2)(x)
x = layers.Dropout(0.3)(x)
x = layers.Conv2D(
    256, kernel_size = 4 , strides = 2 , padding = "same" , use_bias = False
)(x)
x = layers.BatchNormalization(momentum = 0.9)(x)
x = layers.LeakyReLU(0.2)(x)
x = layers.Dropout(0.3)(x)
x = layers.Conv2D(
    512 , kernel_size = 4, strides = 2, padding = "same" , use_bias = False
)(x)
x = layers.BatchNormalization(momentum = 0.9)(x)
x = layers.LeakyReLU(0.2)(x)
x = layers.Dropout(0.3)(x)
x = layers.Conv2D(
    1,
    kernel_size = 4,
    strides = 1,
    padding = "valid",
    use_bias = False,
    activation = "sigmoid",
)(x)
discriminator_output = layers.Flatten()(x)

discriminator = models.Model(discriminator_input, discriminator_output)
discriminator.summary()

In [ ]:
generator_input = layers.Input(shape = (Z_DIM,))
x = layers.Reshape((1,1, Z_DIM))(generator_input)
x = layers.Conv2DTranspose(
    512, kernel_size = 4 , strides = 1 , padding = "valid" , use_bias = False
)(x)
x = layers.BatchNormalization(momentum = 0.9)(x)
x = layers.LeakyReLU(0.2)(x)
x = layers.Conv2DTranspose(
    256, kernel_size = 4, strides =2 , padding = "same", use_bias = False
)(x)
x = layers.BatchNormalization(momentum = 0.9)(x)
x = layers.LeakyReLU(0.2)(x)
x = layers.Conv2DTranspose(
    128, kernel_size = 4 , strides = 2 , padding = "same", use_bias = False
)(x)
x = layers.BatchNormalization(momentum = 0.9)(x)
x = layers.LeakyReLU(0.2)(x)
x = layers.Conv2DTranspose(
    64, kernel_size = 4, strides = 2, padding = "same", use_bias = False
)(x)
x = layers.BatchNormalization(momentum = 0.9)(x)
x = layers.LeakyReLU(0.2)(x)
generator_output = layers.Conv2DTranspose(
    CHANNELS,
    kernel_size = 4,
    strides = 2,
    padding = "same",
    use_bias = False,
    activation = "tanh",
)(x)
generator = models.Model(generator_input, generator_output)
generator.summary()

In [ ]:
class DCGAN(models.Model):
  def __init__(self, discriminator, generator , latent_dim):
    super(DCGAN, self) .__init__()
    self.discriminator = discriminator
    self.generator = generator
    self.latent_dim = latent_dim

  def compile(self, d_optimizer , g_optimzer):
    super(DCGAN, self).compile()
    self.loss_fn = losses.BinaryCrossentropy()
    self.d_optimizer = d_optimizer
    self.g_optimizer = g_optimizer
    self.d_loss_metric = metrics.Mean(name = "d_loss")
    self.d_real_acc_metric = metrics.BinaryCrossentropy(name = "d_real_acc")
    self.d_fake_acc_metric = metrics.BinaryCrossentropy(name = "d_fake_acc")
    self.d_acc_metric = metrics.BinaryAccuracy(name = "d_acc")
    self.g_loss_metric = metrics.Mean(name = "g_loss")
    self.g_acc_metric = metrics.BinaryAccuracy(name = "g_acc")

  @property
  def metrics(self):
    return[
        self.d_loss_metric,
        self.d_real_acc_metric,
        self.d_fake_acc_metric,
        self.d_acc_metric,
        self.g_loss_metric,
        self.g_acc_metric,
    ]

  def train_step(self, real_images):
    #Sample random points in the latent space
    batch_size = tf.shape(real_images)[0]
    random_latent_vectors = tf.random.normal(
        shape=(batch_size, self.latent_dim)
    )

    #Train the discriminator on fake images
    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
      generated_images = self.generator(
          random_latent_vectors, training = True
      )
      real_predictions = self.discriminator(real_images, training = True)
      fake_predictions = self.discriminator(
          generated_images , training = True
      )
      real_labels = tf.ones_like(real_predictions)
      real_noisy_labels = real_labels + NOISE_PARAM * tf.random.uniform(
          tf.shape(real_predictions)
      )
      fake_labels = tf.zeros_like(fake_predictions)
      fake_noisy_labels = fake_labels - NOISE_PARAM * tf.random.uniform(
          tf.shape(fake_predictions)
      )

      d_real_loss = self.loss_fn(real_noisy_labels, real_predictions)
      d_fake_loss = self.loss_fn(fake_noisy_labels, fake_predictions)
      d_loss = (d_real_loss + d_fake_loss) /2.0

      g_loss = self.loss_fn(real_labels , fake_predictions)

    gradients_of_discriminator = dics_tape.gradient(
        d_loss , self.discriminator.trainable_varaibles
    )
    gradient_of_generator = gen_tape.gradient(
        g_loss, self.generator.trainable_varaibles
    )